### ConversationEntityMemory
대화 내용에서 엔티티 정보를 추출하고 저장하는 메모리 클래스입니다
- ConversationEntityMemory는 대화 내용을 저장하면서 동시에 엔티티 정보를 추출하여 별도로 관리합니다

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from dotenv import load_dotenv

In [2]:
# Entity Memory를 사용하는 프롬프트 내용을 출력합니다.
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)


You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

In [3]:
llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature=0)

In [6]:
conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [9]:
conversation.predict(
    input="gong와 gang는 한 학교에서 공부하는 학우입니다."
    "gong과 gang은 lagngchain을 공부하는 주니어 개발자입니다.. "
    "그들은 블로그를 운영하면서 논문을 리뷰하고 최신 ai 기술 동향을 파악하려고 노력하고 있습니다."
)

'그렇군요! gong과 gang이 lagngchain을 공부하고 블로그를 운영하는 것은 정말 흥미로운 일입니다. 그들이 어떤 주제에 대해 논문을 리뷰하고 있는지, 또는 어떤 최신 AI 기술 동향에 관심이 있는지 궁금하네요. 혹시 그들의 블로그에 대해 더 이야기해 줄 수 있나요?'

In [10]:
# entity memory 를 출력합니다.
conversation.memory.entity_store.store


{'gong': 'gong는 한 학교에서 공부하는 학우이며, lagngchain을 공부하는 주니어 개발자로 블로그를 운영하고 논문을 리뷰하고 최신 AI 기술 동향을 파악하려고 노력하고 있습니다.',
 'gang': 'gang은 gong와 함께 한 학교에서 공부하는 학우이며, lagngchain을 공부하는 주니어 개발자입니다. 그들은 블로그를 운영하면서 논문을 리뷰하고 최신 AI 기술 동향을 파악하려고 노력하고 있습니다.',
 'lagngchain': 'gong과 gang은 lagngchain을 공부하는 주니어 개발자입니다.'}

### ConversationKGMemory

대화 내용을 지식 그래프 형태로 저장하고 관리하는 메모리 클래스입니다

ConversationKGMemory는 대화 내용에서 엔티티와 관계를 추출하여 지식 그래프를 구축합니다. 

In [ ]:
from langchain.memory import ConversationKGMemory
##지식 그래프의 힘을 활용하여 정보를 저장하고 불러옵니다.

##이를 통해 모델이 서로 다른 개체 간의 관계를 이해하는 데 도움을 주고, 복잡한 연결망과 역사적 맥락을 기반으로 대응하는 능력을 향상시킵니다.

In [13]:

memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "이쪽은 Gumi 에 거주중인 김강우씨 입니다."},
    {"output": "김강우씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김강우씨는 우리 학교의 langchain 주니어 개발자입니다."},
    {"output": "만나서 반갑습니다."},
)

In [15]:
memory.load_memory_variables({"input": "김강우씨는 누구입니까?"})


{'history': [SystemMessage(content='On 김강우씨: 김강우씨 is a langchain junior developer. 김강우씨 is at our school.', additional_kwargs={}, response_metadata={})]}

In [16]:
from langchain_core.prompts import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

In [17]:
conversation_with_kg.predict(
    input="My name is Teddy. Shirley is a coworker of mine, and she's a new designer at our company."
)


"Hi Teddy! It's great to meet you. How exciting that Shirley is a new designer at your company! What kind of design work does she do? Is she focusing on graphic design, product design, or something else?"

In [18]:
# Shirley 에 대한 질문
conversation_with_kg.memory.load_memory_variables({"input": "who is Shirley?"})


{'history': 'On Shirley: Shirley is a coworker. Shirley is a new designer. Shirley works at the company.'}